In [1]:
import numpy as np
import theano
import theano.tensor as T
import os, pickle, gzip
from PIL import Image

from theano import function
rng = np.random

In [2]:
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
f.close()

In [3]:
def shared_dataset(data_xy):
    data_x, data_y = data_xy
    shared_x = theano.shared(np.asarray(data_x, dtype=theano.config.floatX))
    shared_y = theano.shared(np.asarray(data_y, dtype=theano.config.floatX))

    return shared_x, T.cast(shared_y, 'int32')

test_set_x, test_set_y = shared_dataset(test_set)
valid_set_x, valid_set_y = shared_dataset(valid_set)
train_set_x, train_set_y = shared_dataset(train_set)

batch_size = 500

data = train_set_x[2 * batch_size: 3 * batch_size]
label = train_set_y[2 * batch_size: 3 * batch_size]

In [4]:
class LogisticRegression(object):

    def __init__(self, input, n_in, n_out):

        self.W = theano.shared(
            value=np.zeros(
                (n_in, n_out),
                dtype=theano.config.floatX
            ),
            name='W',
            borrow=True
        )

        self.b = theano.shared(
            value=np.zeros(
                (n_out,),
                dtype=theano.config.floatX
            ),
            name='b',
            borrow=True
        )

        self.p_y_given_x = T.nnet.softmax(T.dot(input, self.W) + self.b)
        self.y_pred = T.argmax(self.p_y_given_x, axis=1)
        self.params = [self.W, self.b]
        self.input = input

    def negative_log_likelihood(self, y):
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])

    def errors(self, y):
        return T.mean(T.neq(self.y_pred, y))

def sgd_optimization_mnist(learning_rate=0.13, n_epoches=1000,
                           batch_size=600):
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size
    n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] // batch_size
    n_test_batches = test_set_x.get_value(borrow=True).shape[0] // batch_size

    index = T.lscalar()

    x = T.matrix('x')
    y = T.ivector('y')

    classifier = LogisticRegression(input=x, n_in=28*28, n_out=10)

    cost = classifier.negative_log_likelihood(y)

    test_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size],
            y: test_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    validate_model = theano.function(
        inputs=[index],
        outputs=classifier.errors(y),
        givens={
            x: valid_set_x[index * batch_size: (index + 1) * batch_size],
            y: valid_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    g_W = T.grad(cost=cost, wrt=classifier.W)
    g_b = T.grad(cost=cost, wrt=classifier.b)

    updates = [(classifier.W, classifier.W - learning_rate * g_W),
               (classifier.b, classifier.b - learning_rate * g_b)]

    train_model = theano.function(
        inputs=[index],
        outputs=cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    epoch = 0
    while epoch < n_epoches:
        epoch = epoch + 1
        for minibatch_index in range(n_train_batches):
            minibatch_avg_cost = train_model(minibatch_index)
            iter = (epoch - 1) * n_train_batches + minibatch_index
    return classifier

In [5]:
classifier = sgd_optimization_mnist()

In [16]:
def predict(classifier):
    predict_model = theano.function(
        inputs=[classifier.input],
        outputs=classifier.y_pred
    )

    predicted_values = predict_model(test_set_x.eval())
    return predicted_values

predicted_value = predict(classifier)
test_y = test_set_y.eval()
np.count_nonzero(predicted_value != test_y)
print(predicted_value.shape[0])


10000


In [27]:
def show_digit(image):
    img = Image.fromarray(np.uint8(image), 'L')
    img.show()

In [32]:
os.chdir('/Users/lzx/OneDrive/Projects/untitled folder')
train_images, train_labels = read_data('train-images-idx3-ubyte', 'train-labels-idx1-ubyte')


In [ ]:
N = len(train_images)
N_row, N_col = np.shape(train_images)
N_feats = N_row * N_col
N_hidden = 768
x = T.matrix('x')
y = T.matrix('y')
W = theano.shared(np.random.randn(N_feats, N_hidden), name='w')
a = theano.shared(np.random.randn(N_hidden,10), name='a')
f = T.dot(a, abs(T.dot(W, x)))# / np.sqrt(N_hidden)
score = T.nnet.softmax(f)
prediction = T.argmax(score)
loss = ((y - f) ** 2).sum()
gW = T.grad(loss, W)

train = theano.function(
    inputs = [x, y],
    outputs = []
)
# b = theano.shared(0., name='w')